In [1]:
import sys
sys.path.append("D:/StateOfStates/")

import pandas as pd
import addfips
import Partisian as Partisian
import CauseOfDeathData
from LifeData import LifeData
import numpy as np
import json
import psycopg2
import json
import CauseOfDeathData
import plotly.graph_objects as go
import plotly.express as px

In [2]:
labels, parents, values= CauseOfDeathData.get_all_age_cod("Florida")

D:/StateOfStates\CauseOfDeathData.py:196: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con)
D:/StateOfStates\CauseOfDeathData.py:196: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con)
D:/StateOfStates\CauseOfDeathData.py:196: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con)
D:/StateOfStates\CauseOfDeathData.py:196: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2

HERE
HERE
HERE
HERE
HERE


D:/StateOfStates\CauseOfDeathData.py:196: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con)
D:/StateOfStates\CauseOfDeathData.py:196: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con)
D:/StateOfStates\CauseOfDeathData.py:196: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con)


### Setup DB connection

In [10]:
def get_db_properties():
    props = {}
    separator = "="
            
    with open(r"C:\Users\zeusg\OneDrive\Documents\Github\habit_final\database.properties") as file:
        for line in file: 
            if separator in line:
                name, value = line.split(separator, 1)
                props[name.strip()] = value.strip()
    return props
db_prop = get_db_properties()
connection = psycopg2.connect(
    host=db_prop["host"],
    user=db_prop["username"],
    password=db_prop["password"],
    database="StateOfStates",
)

### Helper Dicts to convert states->abv->fips

In [ ]:
states =  {'Alaska': 'AK','Alabama': 'AL','Arkansas': 'AR','American Samoa': 'AS','Arizona': 'AZ','California': 'CA','Colorado': 'CO','Connecticut': 'CT','District of Columbia': 'DC','Delaware': 'DE','Florida': 'FL','Georgia': 'GA','Guam': 'GU','Hawaii': 'HI','Iowa': 'IA','Idaho': 'ID','Illinois': 'IL','Indiana': 'IN','Kansas': 'KS','Kentucky': 'KY','Louisiana': 'LA','Massachusetts': 'MA','Maryland': 'MD','Maine': 'ME','Michigan': 'MI','Minnesota': 'MN','Missouri': 'MO','Northern Mariana Islands': 'MP','Mississippi': 'MS','Montana': 'MT','National': 'NA','North Carolina': 'NC','North Dakota': 'ND','Nebraska': 'NE','New Hampshire': 'NH','New Jersey': 'NJ','New Mexico': 'NM','Nevada': 'NV','New York': 'NY','Ohio': 'OH','Oklahoma': 'OK','Oregon': 'OR','Pennsylvania': 'PA','Puerto Rico': 'PR','Rhode Island': 'RI','South Carolina': 'SC','South Dakota': 'SD','Tennessee': 'TN','Texas': 'TX','Utah': 'UT','Virginia': 'VA','Virgin Islands': 'VI','Vermont': 'VT','Washington': 'WA','Wisconsin': 'WI','West Virginia': 'WV','Wyoming': 'WY'}
states_abv = {v: k for k, v in states.items()}
af = addfips.AddFIPS()


### Load Partisian DF

In [ ]:
#State Control
partisian_df = Partisian.get_state_part_score_all()
partisian_df = partisian_df.rename(columns={"State": "State_Code"})
#State Names
states_df = partisian_df[["STATE", "State_Code", "fips"]].drop_duplicates().copy()
states_df["level"] = "State"
states_df = states_df.rename(columns={"STATE": "name", "State_Code": "acronym"})
states_df["parent"] = 1
states_df = states_df[["level", "name", "parent", "fips", "acronym"]]
#County Votes
county_votes_df = Partisian.get_county_votes()
## TODO - Clean up county votes




In [ ]:
def insert_states(df):
    cursor = connection.cursor()
    for index, row in df.iterrows():
        cursor.execute("INSERT INTO public.locations (level, name, parent, fips, acronym) \
                       VALUES (%s, %s, %s, %s, %s);", 
                       (row["level"], row["name"], row["parent"], row["fips"], row["acronym"]))
    connection.commit()
    cursor.close()
    return "success"
#insert_states(states_df)


In [ ]:
def insert_partisian_control(df):
    cursor = connection.cursor()
    for index, row in df.iterrows():
        cursor.execute("SELECT id FROM public.locations WHERE acronym = %s;", (row["State_Code"],))
        locations_fk = cursor.fetchone()[0]
        cursor.execute("INSERT INTO public.partisans (year, control_score, locations_id_fkey) \
            VALUES (%s, %s, %s);", 
            (row["Year"], row["Part_Score"], locations_fk))
    connection.commit()
    cursor.close()
    return "success"
#insert_partisian_control(partisian_df)

### Load ICD10 codes

In [ ]:
icd10 = CauseOfDeathData.get_heirarchy_df().T.reset_index()
icd10 = icd10.rename({"index": "name"}, axis=1)

### Insert ICD10 and Hierarchy structure

In [ ]:
def insert_inital_icd10(df):
    cursor = connection.cursor()
    for index, row in df.iterrows():
        cursor.execute("INSERT INTO public.icd_10_codes (name, common_name) \
            VALUES (%s, %s);", 
            (row["name"], row["common_name"]))
    connection.commit()
    cursor.close()
    return "success"
def add_icd10_hierarchy(df):
    cursor = connection.cursor()
    for index, row in df.iterrows():
        if len(row["parents"]) > 0:
            cursor.execute("SELECT id FROM public.icd_10_codes WHERE name = %s;", (row["parents"][0],))
            icd10_fk = cursor.fetchone()[0]
            cursor.execute("UPDATE public.icd_10_codes SET parent = %s WHERE name = %s;", 
                (icd10_fk, row["name"]))
    connection.commit()
    cursor.close()
    return "success"

def add_icd10_codes(df):
    cursor = connection.cursor()
    for index, row in df.iterrows():
        codes_json = json.dumps(row["ranges"])
        cursor.execute("UPDATE public.icd_10_codes SET codes = %s WHERE name = %s;", 
            (codes_json, row["name"]))
    connection.commit()
    cursor.close()
    ## TODO - Cleaning needed when code spans chapters IE: A00-B99
    return "success"
    
#insert_inital_icd10(icd10)
#add_icd10_hierarchy(icd10)
#add_icd10_codes(icd10)

### Load Cause Of Death for all states

In [ ]:
cod = CauseOfDeathData.load_state_cod(state="All")
cod["year"] = 2019
cod = cod[["Rate", "Deaths", "Population","year","ICD-10 113 Cause List","State","Ten-Year Age Groups Code", "Crude Rate"]]
cod = cod[cod["State"] != "District of Columbia"]

In [ ]:
def add_cod(df):
    try:
        cursor = connection.cursor()
        for index, row in df.iterrows():
            cursor.execute("SELECT id FROM public.locations WHERE name = %s;", (row["State"],))
            state_id = cursor.fetchone()[0]
            cursor.execute("SELECT id FROM public.icd_10_codes WHERE name = %s;", (row["ICD-10 113 Cause List"],))
            icd10_id = cursor.fetchone()[0]
            if row["Ten-Year Age Groups Code"] == "1":
                start_age = 0
                end_age = 1
            elif row["Ten-Year Age Groups Code"] == "85+":
                start_age = 85
                end_age = 200
            else:
                start_age = int(row["Ten-Year Age Groups Code"].split("-")[0])
                end_age = int(row["Ten-Year Age Groups Code"].split("-")[1])
            unreliable = False
            if row["Crude Rate"] == "Unreliable":
                unreliable = True
            cursor.execute("INSERT INTO public.causes_of_death (rate, deaths, population, year, locations_id_fkey, icd_10_codes_id_fkey, start_age, end_age, unreliable) \
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);", 
                (row["Rate"], row["Deaths"], row["Population"], row["year"], state_id, icd10_id, start_age, end_age, unreliable))

        connection.commit()
    except Exception as e:
        print(row["State"])
        print(e)
    finally:
        cursor.close()
    return "success"
add_cod(cod)

In [8]:
sql = "select l.name as state, l.acronym, l.fips, \
cod.rate, cod.deaths, cod.population, cod.start_age, cod.end_age, \
ic.name, ic.common_name, \
ic2.name as parent_name, ic2.common_name as parent_common_name \
from causes_of_death cod \
inner join icd_10_codes ic on cod.icd_10_codes_id_fkey = ic.id \
left join icd_10_codes ic2 on ic.parent = ic2.id \
inner join locations l ON cod.locations_id_fkey = l.id \
where l.level = 'State'"

In [ ]:
df = pd.read_sql(sql, connection)

In [ ]:
test = list(df.start_age.unique())
test.sort()
test

In [ ]:
df = df[df["state"] == "Alabama"]
df = df[df["start_age"] == 55]
df.sample(3)

In [ ]:
#Calculate sum of parents by rate of it's children
for name in df["parent_name"].unique():
    parent_df = df[df["parent_name"] == name]
    parent_rate = parent_df["rate"].sum()
    original_rate = 0
    try:
        orignal_rate = df[df["name"] == name]["rate"].values[0]
    except:
        pass
    if parent_rate > orignal_rate:
        df.loc[df["name"] == name, "rate"] = parent_rate
    
    

In [ ]:
old_df = CauseOfDeathData.load_state_cod(state="All")
#old_df

In [ ]:
names = list(df["common_name"])
#parents = [""]
parents= list(df["parent_common_name"])
parents = ['' if v is None else v for v in parents]
values = list(df["rate"])

In [ ]:
fig = px.treemap(
    branchvalues = "total",
    names = names,
    parents = parents,
    values = values,
)
fig.update_traces(root_color="lightgrey")
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

In [ ]:
gap =  px.data.gapminder().query("year == 2007")
gap

In [ ]:
labels, parents, values, df_age, total = CauseOfDeathData.get_age_cod_db(55, False, "Alabama")

In [ ]:
fig = px.treemap(
    branchvalues = "total",
    names = labels,
    parents = parents,
    values = values,
)
fig.update_traces(root_color="lightgrey")
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

In [3]:
labels, parents, values= CauseOfDeathData.get_all_age_cod("Florida")

D:/StateOfStates\CauseOfDeathData.py:196: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con)
D:/StateOfStates\CauseOfDeathData.py:196: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con)
D:/StateOfStates\CauseOfDeathData.py:196: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con)
D:/StateOfStates\CauseOfDeathData.py:196: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2

HERE
HERE
HERE
HERE
HERE


D:/StateOfStates\CauseOfDeathData.py:196: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con)
D:/StateOfStates\CauseOfDeathData.py:196: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con)
D:/StateOfStates\CauseOfDeathData.py:196: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con)


In [13]:
len(values)

714

In [23]:
labels[360]

'Other Heart Diseases: 55-64'

In [21]:
parents[360]

'Heart Disease: 55-64'

In [18]:
values[360]

42.1

In [11]:
df = pd.read_sql(sql, connection)
fl = df[df["state"] == "Florida"]
fl = fl[fl["start_age"]== 55] 
fl[fl["common_name"] == "Heart failure"]

C:\Users\zeusg\AppData\Local\Temp/ipykernel_23848/3558574259.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, connection)


,state,acronym,fips,rate,deaths,population,start_age,end_age,name,common_name,parent_name,parent_common_name
11013,Florida,FL,12,9.01,261,2898355,55,64,Heart failure (I50),Heart failure,Other heart diseases (I26-I51),Other Heart Diseases


In [25]:

fl[fl["common_name"] == "Heart Disease"]

,state,acronym,fips,rate,deaths,population,start_age,end_age,name,common_name,parent_name,parent_common_name
8111,Florida,FL,12,179.79,5211,2898355,55,64,"#Diseases of heart (I00-I09,I11,I13,I20-I51)",Heart Disease,Major cardiovascular diseases (I00-I78),Heart Disease (Major)


In [24]:
fl[fl["parent_common_name"] == "Heart Disease"].rate.sum()

179.79

In [37]:
parents[360]
values[360] = values[360]-1

In [5]:
fig = px.treemap(
    branchvalues = "total",
    names = labels,
    parents = parents,
    values = values,
)
fig.update_traces(root_color="lightgrey")
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

In [20]:
fig = go.Figure(go.Treemap(
        branchvalues = "total",
        labels = labels,
        parents =  parents,
        values=values,
        textinfo = "label+value+percent entry",
        textposition = 'middle center',
        textfont = dict(family="Arial", size=16, color = '#FFFFFF'),
        pathbar_textfont_size=15,
        pathbar_visible=True,
    ))
fig.show()